In [1]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import cv2
import io

In [2]:
tensorrt_model = YOLO("/mnt/data1/model_1122/image_detection/best.engine", task="detect")

In [15]:
def process_predict(results):
    gender_set = {"雄性", "雌性"}
    age_set = {'成体', '亚成体', "幼体"}
    detect_class = []
    agg_detect_class = []
    classs_id_2_num = {}
    class_id_2_detail_dict = {}
    for result in results:
        # 获取边界框、置信度和类别
        boxes = result.boxes
        # 遍历每个检测框
        for box in boxes:
            # 获取边界框坐标 (x1, y1, x2, y2)
            x1, y1, x2, y2 = box.xyxy[0].tolist()  # xyxy格式
            # 获取置信度分数
            confidence = box.conf[0].item()
            # 获取预测类别的索引
            class_id = int(box.cls[0].item())
            if class_id not in classs_id_2_num:
                classs_id_2_num[class_id] = 0
            classs_id_2_num[class_id] += 1
            # 获取类别名称（如果模型中有类别名称映射）
            class_name = result.names[class_id]
            
            class_name_copy = class_name
            gender = "无法区分"
            for gd in gender_set:
                if gd in class_name_copy:
                    class_name_copy = class_name_copy.replace(gd, "")
                    gender = gd
            age = "无法区分"
            for ag in age_set:
                if ag in class_name_copy:
                    class_name_copy = class_name_copy.replace(ag, "")
                    age = ag
            class_id_2_detail_dict[class_id] = [class_name_copy, gender, age]
            detect_class.append((class_name, (x1, y1, x2, y2), confidence, [class_name_copy, gender, age]))
            print(f"检测到 {class_name} - 置信度: {confidence:.2f}")
            print(f"位置: x1={x1:.1f}, y1={y1:.1f}, x2={x2:.1f}, y2={y2:.1f}")
    for k, v in classs_id_2_num.items():
        agg_detect_class.append((class_id_2_detail_dict[k], v))

    return {'results': detect_class, 'agg_results': agg_detect_class}
    

In [21]:
image_path = "/mnt/data1/model_1122/image_classifier/test/animal/YNNR008X-WD0014-20161208-03039.jpg"
with open(image_path, 'rb') as file:
    image_content = file.read()
image_path = Image.open(io.BytesIO(image_content))
results = tensorrt_model.predict(image_path, conf=0.256)
# 遍历每张图片的结果（如果只有一张图片，results[0]就是那张图片的结果）
process_predict(results)


0: 640x640 3 水牛成体s, 36.9ms
Speed: 3.0ms preprocess, 36.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
检测到 水牛成体 - 置信度: 0.48
位置: x1=856.2, y1=1525.2, x2=1443.3, y2=1829.2
检测到 水牛成体 - 置信度: 0.32
位置: x1=1465.1, y1=786.4, x2=2552.2, y2=1359.7
检测到 水牛成体 - 置信度: 0.26
位置: x1=329.0, y1=1062.1, x2=1189.2, y2=1402.5


{'results': [('水牛成体',
   (856.1932983398438, 1525.155029296875, 1443.26904296875, 1829.16845703125),
   0.4818171560764313,
   ['水牛', '无法区分', '成体']),
  ('水牛成体',
   (1465.12353515625, 786.4356689453125, 2552.24169921875, 1359.7164306640625),
   0.3194273114204407,
   ['水牛', '无法区分', '成体']),
  ('水牛成体',
   (328.9973449707031, 1062.0830078125, 1189.159912109375, 1402.52880859375),
   0.2574602961540222,
   ['水牛', '无法区分', '成体'])],
 'agg_results': [(['水牛', '无法区分', '成体'], 3)]}